In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import pandas as pd

In [3]:
from stellargraph import StellarGraph
from stellargraph.data import UnsupervisedSampler, EdgeSplitter, BiasedRandomWalk

2022-02-05 13:15:55.703212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 13:15:55.711580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 13:15:55.711808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 13:15:55.712436: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

# Load the data

In [4]:
from shared.schema import DatasetSchema, GraphSchema
from shared.graph.loading import pd_from_entity_schema


In [5]:
DATASET = DatasetSchema.load_schema('star-wars')
schema = GraphSchema.from_dataset(DATASET)

In [6]:
explicit_label = False
explicit_timestamp = True
unix_timestamp = True
prefix_id = None
include_properties = lambda cs: [c for c in cs if c.startswith('feat_') or c == 'name']

nodes_dfs = {
    label: pd_from_entity_schema(
        entity_schema,
        explicit_label=explicit_label,
        explicit_timestamp=explicit_timestamp,
        include_properties=include_properties,
        unix_timestamp=unix_timestamp,
        prefix_id=prefix_id,
    ).set_index('id').drop(columns=['type']).sort_index()
    for label, entity_schema in schema.nodes.items()
}

edges_dfs = {
    label: pd_from_entity_schema(
        entity_schema,
        explicit_label=explicit_label,
        explicit_timestamp=explicit_timestamp,
        include_properties=include_properties,
        unix_timestamp=unix_timestamp,
        prefix_id=prefix_id,
    ).reset_index().drop(columns=['type']).drop_duplicates(subset=['src', 'dst', 'timestamp'])
    for label, entity_schema in schema.edges.items()
}

cursor = 0
for df in edges_dfs.values():
    df.index += cursor
    cursor += len(df)

In [7]:
graph = StellarGraph(
    nodes={k: df.drop(columns=['name_']) for k, df in nodes_dfs.items()},
    edges=edges_dfs,
    source_column='src',
    target_column='dst',
)
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 2078

 Node types:
  Character: [113]
    Features: float32 vector, length 32
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [1120]
        Weights: all 1 (default)
        Features: float32 vector, length 2
    Character-INTERACTIONS->Character: [958]
        Weights: all 1 (default)
        Features: float32 vector, length 2


# Split dataset

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
edge_splitter_test = EdgeSplitter(graph)
graph_sub_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.05, method="global"
)

print(graph_sub_test.info())

** Sampled 103 positive and 103 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 1975

 Node types:
  Character: [113]
    Features: float32 vector, length 32
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [1066]
        Weights: all 1 (default)
        Features: none
    Character-INTERACTIONS->Character: [909]
        Weights: all 1 (default)
        Features: none


In [10]:
train_size = 0.75
val_size = 0.25

edge_splitter_train = EdgeSplitter(graph_sub_test)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)

(
    examples_train,
    examples_val,
    labels_train,
    labels_val,
) = train_test_split(examples, labels, train_size=train_size, test_size=val_size)

print(graph_train.info())

** Sampled 197 positive and 197 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 1778

 Node types:
  Character: [113]
    Features: float32 vector, length 32
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [954]
        Weights: all 1 (default)
        Features: none
    Character-INTERACTIONS->Character: [824]
        Weights: all 1 (default)
        Features: none


In [11]:
pd.DataFrame(
    [
        (
            "Training Set",
            len(examples_train),
            "Train Graph",
            "Test Graph",
            "Train the Link Classifier",
        ),
        (
            "Validation Set",
            len(examples_val),
            "Train Graph",
            "Test Graph",
            "Validate the Link Classifier",
        ),
        (
            "Test set",
            len(examples_test),
            "Test Graph",
            "Full Graph",
            "Evaluate Link Classifier",
        ),
    ],
    columns=("Split", "Number of Examples", "Hidden from", "Picked from", "Use"),
).set_index("Split")


,Number of Examples,Hidden from,Picked from,Use
Split,,,,
Training Set,295,Train Graph,Test Graph,Train the Link Classifier
Validation Set,99,Train Graph,Test Graph,Validate the Link Classifier
Test set,206,Test Graph,Full Graph,Evaluate Link Classifier


# Train Attri2Vec

In [12]:
from tensorflow import keras

from stellargraph.mapper import Attri2VecLinkGenerator
from stellargraph.layer import Attri2Vec, link_classification

In [13]:
batch_size = 30
epochs = 100
walk_length = 5
walk_number = 2

In [14]:
unsupervised_samples = UnsupervisedSampler(
    graph_train, nodes=list(graph_train.nodes()), length=walk_length, number_of_walks=walk_number
)

generator = Attri2VecLinkGenerator(graph_train, batch_size)

In [15]:
layer_sizes = [128]

hinsage = Attri2Vec(
    layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None
)

x_inp, x_out = hinsage.in_out_tensors()
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [16]:
model = keras.Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          4096        input_1[0][0]                    
__________________________________________________________________________________________________
output_embedding (Embedding)    (None, 1, 128)       14464       input_2[0][0]                    
______________________________________________________________________________________________

In [18]:
model.fit(
    generator.flow(unsupervised_samples),
    validation_data=generator.flow(examples_val, labels_val),
    epochs=epochs,
    verbose=2,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True
        )
    ],
)

Epoch 1/100


2022-02-05 13:15:57.263821: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-05 13:15:58.354226: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


61/61 - 2s - loss: 0.7017 - binary_accuracy: 0.4967 - val_loss: 0.6727 - val_binary_accuracy: 0.5859
Epoch 2/100
61/61 - 0s - loss: 0.6940 - binary_accuracy: 0.5448 - val_loss: 0.6687 - val_binary_accuracy: 0.5859
Epoch 3/100
61/61 - 0s - loss: 0.6884 - binary_accuracy: 0.5675 - val_loss: 0.6726 - val_binary_accuracy: 0.6263
Epoch 4/100
61/61 - 0s - loss: 0.6963 - binary_accuracy: 0.5509 - val_loss: 0.6665 - val_binary_accuracy: 0.6364
Epoch 5/100
61/61 - 0s - loss: 0.7007 - binary_accuracy: 0.5459 - val_loss: 0.6668 - val_binary_accuracy: 0.5960
Epoch 6/100
61/61 - 0s - loss: 0.7029 - binary_accuracy: 0.5183 - val_loss: 0.6862 - val_binary_accuracy: 0.5657
Epoch 7/100
61/61 - 0s - loss: 0.6919 - binary_accuracy: 0.5465 - val_loss: 0.6788 - val_binary_accuracy: 0.5758
Epoch 8/100
61/61 - 0s - loss: 0.6907 - binary_accuracy: 0.5332 - val_loss: 0.6717 - val_binary_accuracy: 0.6263
Epoch 9/100
61/61 - 0s - loss: 0.6896 - binary_accuracy: 0.5492 - val_loss: 0.6900 - val_binary_accuracy: 0.

# Extract and cluster the embeddings

In [19]:
embedding_model = keras.Model(inputs=x_inp, outputs=x_out)

In [20]:
graph_node_list = graph.nodes()

node_embeddings = embedding_model.predict(
    generator.flow(list(zip(graph_node_list, graph_node_list)))
)
node_embeddings = node_embeddings[0]

In [21]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0).fit(node_embeddings)

In [22]:
from shared.constants import BENCHMARKS_RESULTS

save_path = BENCHMARKS_RESULTS.joinpath('analysis', 'node2vec-linkprediction-kmeans')
save_path.mkdir(parents=True, exist_ok=True)

In [23]:
from shared.graph import CommunityAssignment

labeling = pd.Series(kmeans.labels_, index=graph_node_list, name="cid")
comlist = CommunityAssignment(labeling)

In [24]:
comlist.save_comlist(save_path.joinpath('schema.comlist'))

In [ ]:
from datasets.scripts import export_to_visualization

export_to_visualization.run(
    export_to_visualization.Args(
        dataset='star-wars',
        version='base',
        run_paths=[str(save_path)]
    )
)

[2022-02-05 13:16:46,688][export_to_visualization.py][INFO] Processing /data/pella/projects/University/Thesis/Thesis/code/storage/datasets/processed/star-wars/schema.yaml
ERROR! Session/line number was not unique in database. History logging moved to new session 48
[2022-02-05 13:16:46,760][export_to_visualization.py][INFO] Adding prediction /data/pella/projects/University/Thesis/Thesis/code/storage/results/analysis/node2vec-linkprediction-kmeans/schema.comlist as prediction_0
[2022-02-05 13:16:46,765][export_to_visualization.py][INFO] Writing /data/pella/projects/University/Thesis/Thesis/code/storage/datasets/export/star-wars/versions/base/schema.graphml
